I used those commands to connect to my google drive in order to save and retrieve the preprocessed data without starting from scratch


In [3]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [19]:
%cd ../gdrive/MyDrive/bigdata


/gdrive/MyDrive/bigdata


In [20]:
!ls


amazon-fine-food-reviews.zip  hashes.txt   Reviews.csv	X.pkl
database.sqlite		      kaggle.json  reviews.txt


In [5]:
!pip install pickle-mixin --user

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pickle-mixin: filename=pickle_mixin-1.0.2-py3-none-any.whl size=6008 sha256=0167e1fea567cd79354c0e4a7f516283cad58113491f6f41cf54b3588df25976
  Stored in directory: /root/.cache/pip/wheels/d0/70/0b/673e09a7ed429660d22352a1b117b4f616a8fc054bdd7eb157
Successfully built pickle-mixin


In [6]:
import pandas as pd
import nltk
import re
import string
import random 
import numpy as np
import pickle
import scipy

used this to insert my kaggle key to download the dataset from kaggle without uploading it

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content"

In [ ]:
!chmod 600 /content/kaggle.json

In [ ]:
!kaggle datasets download -d snap/amazon-fine-food-reviews

In [ ]:
from zipfile import ZipFile

In [ ]:
# Create a ZipFile Object and load sample.zip in it
with ZipFile('amazon-fine-food-reviews.zip', 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall()

i used the preprocessed procedure linked in the classroom. I basically remove stop words and i transform everything into tfidf vectors.

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')

from nltk.tokenize import word_tokenize #Used to extract words from documents
from nltk.stem import WordNetLemmatizer #Used to lemmatize words
from sklearn.feature_extraction.text import TfidfVectorizer

read dataset and shuffle it so that it is easier to use just a subset of it to do testings.

In [ ]:
#read data from csv and get the reviews
df = pd.read_csv('Reviews.csv')
reviews=df["Text"].tolist()
#shuffle to later divide in test and training without bias
random.shuffle(reviews)

remove contraddictions and abbreviations to simplify the analysis of documents

In [ ]:
#preprocess the reviews
contractions_dict = {
  "ain't": "am not",
  "aren't": "are not",
  "can't": "cannot",
  "can't've": "cannot have",
  "'cause": "because",
  "could've": "could have",
  "couldn't": "could not",
  "couldn't've": "could not have",
  "didn't": "did not",
  "doesn't": "does not",
  "don't": "do not",
  "hadn't": "had not",
  "hadn't've": "had not have",
  "hasn't": "has not",
  "haven't": "have not",
  "he'd": "he would",
  "he'd've": "he would have",
  "he'll": "he will",
  "he'll've": "he will have",
  "he's": "he is",
  "how'd": "how did",
  "how'd'y": "how do you",
  "how'll": "how will",
  "how's": "how is",
  "i'd": "I would",
  "i'd've": "I would have",
  "i'll": "I will",
  "i'll've": "I will have",
  "i'm": "I am",
  "i've": "I have",
  "isn't": "is not",
  "it'd": "it had",
  "it'd've": "it would have",
  "it'll": "it will",
  "it'll've": "it will have",
  "it's": "it is",
  "let's": "let us",
  "ma'am": "madam",
  "mayn't": "may not",
  "might've": "might have",
  "mightn't": "might not",
  "mightn't've": "might not have",
  "must've": "must have",
  "mustn't": "must not",
  "mustn't've": "must not have",
  "needn't": "need not",
  "needn't've": "need not have",
  "o'clock": "of the clock",
  "oughtn't": "ought not",
  "oughtn't've": "ought not have",
  "shan't": "shall not",
  "sha'n't": "shall not",
  "shan't've": "shall not have",
  "she'd": "she would",
  "she'd've": "she would have",
  "she'll": "she will",
  "she'll've": "she will have",
  "she's": "she is",
  "should've": "should have",
  "shouldn't": "should not",
  "shouldn't've": "should not have",
  "so've": "so have",
  "so's": "so is",
  "that'd": "that would",
  "that'd've": "that would have",
  "that's": "that is",
  "there'd": "there had",
  "there'd've": "there would have",
  "there's": "there is",
  "they'd": "they would",
  "they'd've": "they would have",
  "they'll": "they will",
  "they'll've": "they will have",
  "they're": "they are",
  "they've": "they have",
  "to've": "to have",
  "wasn't": "was not",
  "we'd": "we had",
  "we'd've": "we would have",
  "we'll": "we will",
  "we'll've": "we will have",
  "we're": "we are",
  "we've": "we have",
  "weren't": "were not",
  "what'll": "what will",
  "what'll've": "what will have",
  "what're": "what are",
  "what's": "what is",
  "what've": "what have",
  "when's": "when is",
  "when've": "when have",
  "where'd": "where did",
  "where's": "where is",
  "where've": "where have",
  "who'll": "who will",
  "who'll've": "who will have",
  "who's": "who is",
  "who've": "who have",
  "why's": "why is",
  "why've": "why have",
  "will've": "will have",
  "won't": "will not",
  "won't've": "will not have",
  "would've": "would have",
  "wouldn't": "would not",
  "wouldn't've": "would not have",
  "y'all": "you all",
  "y'alls": "you alls",
  "y'all'd": "you all would",
  "y'all'd've": "you all would have",
  "y'all're": "you all are",
  "y'all've": "you all have",
  "you'd": "you had",
  "you'd've": "you would have",
  "you'll": "you will",
  "you'll've": "you will have",
  "you're": "you are",
  "you've": "you have"
}

# Regular expression for finding contractions
def multiple_replace(dict, text):
  # Create a regular expression  from the dictionary keys
  regex = re.compile("(%s)" % "|".join(map(re.escape, dict.keys())))

  # For each match, look-up corresponding value in dictionary
  return regex.sub(lambda mo: dict[mo.string[mo.start():mo.end()]], text)

reviews = [multiple_replace(contractions_dict, doc) for doc in reviews]
table = str.maketrans('', '', string.punctuation)
reviews = [doc.translate(table) for doc in reviews]
reviews = [re.sub(r'\d+', " ", doc) for doc in reviews]

lemmatize the document in order to group words that mean the same thing and so on to have an analysis on the semantic

In [ ]:
def get_lemmatized(doc):
  word_list = word_tokenize(doc)
  lemmatized_doc = ""
  for word in word_list:
    lemmatized_doc = lemmatized_doc + " " + lemmatizer.lemmatize(word)
  return lemmatized_doc

lemmatizer = WordNetLemmatizer()

reviews = [get_lemmatized(doc) for doc in reviews]

remove stopwords that don't capture things that are useful

In [ ]:
stopwords = nltk.corpus.stopwords.words('english') # Returns a list
stopwords = set(stopwords) # We want a set, because this is implemented with a hash table
                           # Checking the if condition in rem_stop costs O(1) in this way

def rem_stop(doc):
   word_list = word_tokenize(doc)
   cleaned_doc = ""
   for word in word_list:
     if word not in stopwords:
       cleaned_doc += " " + word
   return cleaned_doc


reviews = [rem_stop(doc) for doc in reviews]

vectorize the documents with tfidf to capture informations based on the frequencies of the words in the document.Not frequently used words are more useful.

In [ ]:
vectorizer = TfidfVectorizer(stop_words='english', min_df=10) ## Corpus is in English
X = vectorizer.fit_transform(reviews)

used this to save reviews without redoing everything

In [ ]:
# #write reviews to file
with open('reviews.txt', 'w',encoding='utf8') as f:
    for item in reviews:
        f.write(item+",")

save the processed document on the drive to reload it without redo everything everytime i disconnect the environment

In [ ]:
import pickle
#write x to file
with open('X.pkl', 'wb') as f:
	pickle.dump(X, f)

# read obj from file
with open('X.pkl', 'rb') as f:
	obj = pickle.load(f)

this function reads the preprocessed documents saved previously 

In [8]:
import pickle
def readReviewsX():
  #read reviews from file
  with open('reviews.txt', 'r',encoding='utf8') as f:
      reviews = f.read().split(",")
      reviews = reviews[:-1]
  #read X from file
  with open('X.pkl', 'rb') as f:
      X = pickle.load(f)
  return reviews,X


take just a percentage of the dataset to test everything on a small subset

In [9]:
def percentageOfDataset(reviews,X,percentage=1):
  #take just a percentage of the dataset
  reviewsOut=reviews[:int(len(reviews)*percentage)]
  XOut=X[:int(X.shape[0]*percentage)]
  return reviewsOut,XOut


this class contains some operations that are useful both for the normal lsh and for the lsh vectorized

In [10]:
class Operations: 
    #generate a random vector direction used to calculate the hashes saved in the matrix m
    def generateU():
        u=np.random.normal(0,1,cols)
        u=scipy.sparse.csr_matrix(u)
        return u
    #sparse dot product for sparse sci py matrices
    def sparseProduct(x,y):
        return x.dot(y.transpose()).toarray()[0][0]
    #given u and a vector it returns the hash to be saved in the matrix m
    def h(x,u):
        return np.sign(Operations.sparseProduct(x,u))
    #returns the exact cosine similarity of 2 vectors
    def cosineSimilarity(x,y):
        r=Operations.sparseProduct(x,y)/(np.linalg.norm(x.toarray())*np.linalg.norm(y.toarray()))
        return r
    #returns the exact jaccard similarity of 2 vectors
    def jaccardSimilarity(x,y):
        intersection=len(x.intersection(y))
        union=len(x.union(y))
        return intersection/union

this is the hash class used to generate an hash function to be called on an input.
generate nBytes vectors, multiply it for the input and the resulting vector will have 1 if the elements are positive or 0 if <= 0.
This hash is used in the normal lsh not in the vectorized lsh that i wrote

In [11]:
class Hash:
    def __init__(self,nBytes=50,inputDimension=10):
        self.nBytes=nBytes
        self.inputDimension=inputDimension
        self.reset()
    #reset the matrix to generate the hash
    def reset(self):
        self.matrix=np.random.randn(self.nBytes,self.inputDimension)
    #generate the hash as described above
    def generateHash(self,input):
        if type(input)==list:
            input=np.array(input)
        bools = (np.dot(input, self.matrix.T) > 0).astype('int')
        return ''.join(bools.astype('str'))

In [12]:
h=Hash()
h.generateHash(np.ones(10))

'01011010011001011100010001110101101010111111001101'

this class is responsible to perform the generation of the m matrix both for the train and for the test datasets , perform the lsh and compare the results for the approximated results and the exact results

In [53]:

class LshNormal:
    def __init__(self,X,reviews,b=10,r=10,testPercentage=0.005,nHashes=100):
        self.X=X
        self.reviews=reviews
        self.rows=X.shape[0]
        self.cols=X.shape[1]
        self.b=b
        self.r=r
        self.testPercentage=testPercentage
        self.nHashes=nHashes
        self.splitDataset()

    def setBR(self,b,r):
        self.b=b
        self.r=r
    #split the dataset in train and test based on percentage
    def splitDataset(self):
        indexSplit=int(self.rows*self.testPercentage)
        self.testReviews,self.trainReviews=self.reviews[:indexSplit],self.reviews[indexSplit:]
        self.testX,self.trainX=self.X[:indexSplit],self.X[indexSplit:]
        self.rowsTest,self.colsTest=self.testX.shape
        self.rowsTrain,self.colsTrain=self.trainX.shape
    #fucntion to generate the matrix of signatures 
    def generateM(self):
        self.mTest=[[]for i in range(self.rowsTest)]
        self.mTrain=[[]for i in range(self.rowsTrain)]
        for i in range(self.nHashes):
            #generate a random vector u
            u=Operations.generateU()
            for j in range(self.rowsTest):
                #get the hash for every test row
                h=Operations.h(self.testX[j],u)
                self.mTest[j].append(h)
            for j in range(self.rowsTrain):
                #get the hash for every train row
                h=Operations.h(self.trainX[j],u)
                self.mTrain[j].append(h)
    #function to exectute the lsh algorithm
    #starting from the m matrix hash r elements at the time for a total of b times and insert the index in the relative bucket
    def lsh(self):
        self.dicts=[{} for i in range(self.b)]
        #create hash functions
        self.hashesUsed=[Hash(inputDimension=self.r) for i in range(self.b)]
        #for every b
        for i in range(self.b):
        #for every row of the training set
            for j in range(self.rowsTrain):
                #get the r values based on b and hash them 
                v=self.mTrain[j][i*self.r:(i+1)*self.r]
                key=self.hashesUsed[i].generateHash(v)
                #add the index to the hashes
                if key not in self.dicts[i]:
                    self.dicts[i][key]=set()
                self.dicts[i][key].add(j)
    #Ffunction to find the exact cosine similarity
    def findExactCosineSimilars(self,threshold=0.5):
        self.exactSimilars={}
        #for every test row
        for i,v in enumerate(self.testX):
          #for every train rows
            for j,u in enumerate(self.trainX):
              #find exact cosine similarity between the corresponding vectors and if greater than threshold save the result
                r=Operations.cosineSimilarity(v,u)
                if r>=threshold:
                    if i not in self.exactSimilars:
                        self.exactSimilars[i]=set()
                    self.exactSimilars[i].add(j)
        return self.exactSimilars
    #find the cosine similars starting from the buckets created by the lsh function
    def findCosineSimilarsApproximated(self):
        self.approximatedSimilars={}
        #for every test row of m
        for i,v in enumerate(self.mTest):
            #for every hash functions used to do the lsh
            for j,hash in enumerate(self.hashesUsed):
              #key is the signature of the b th block of r values 
                key=hash.generateHash(v[j*self.r:(j+1)*self.r])
                if key in self.dicts[j]:
                  #add add all the elements of the bucket where its been hashed to the set of approximated similars
                    for k in self.dicts[j][key]:
                        if i not in self.approximatedSimilars:
                            self.approximatedSimilars[i]=set()
                        self.approximatedSimilars[i].add(k)
        return self.approximatedSimilars

    def findCosineSimilarsApproximatedSingular(self,i):
        similars=set()
        #for every hash functions used to do the lsh
        for j,hash in enumerate(self.hashesUsed):
          #key is the signature of the b th block of r values 
            key=hash.generateHash(self.mTest[i][j*self.r:(j+1)*self.r])
            if key in self.dicts[j]:
              #add add all the elements of the bucket where its been hashed to the set of approximated similars
                for k in self.dicts[j][key]:
                    similars.add(k)
        return similars
    #function to evaluate the jaccard similarity between the true cosine similars and the appoximated cosine similars
    def jaccardAB(self):
        return Operations.jaccardSimilarity(self.A,self.B)
    def createAB(self):
        self.A=set()
        self.B=set()
        for i in self.exactSimilars:
            for j in self.exactSimilars[i]:
                self.A.add((i,j))
        for i in self.approximatedSimilars:
            for j in self.approximatedSimilars[i]:
                self.B.add((i,j))
    #function i wrote to find the r and b that yields to the best jaccard similarity given the cosine similarity and the number of hashes
    def optimizeRB(self,cosineSimilarity=0.2,nHashes=10):
      self.findExactCosineSimilars(cosineSimilarity)
      self.nHashes=nHashes
      maxTaken=0
      #generate the matrix m
      lsh.generateM()
      #automatic search for b and r
      maxJaccard=0
      maxR=0
      maxB=0
      #try all the r and b until the squared root of the number of hashes try a b and b a
      for i in range(1,int(lsh.nHashes**0.5)):
        lsh.r=i
        lsh.b=int(lsh.nHashes/i)
        lsh.lsh()
        b=lsh.findCosineSimilarsApproximated()
        jacc=lsh.jaccardAB()
        #if global optimum save values
        if jacc>maxJaccard:
          maxJaccard=jacc
          maxR=lsh.r
          maxB=lsh.b
          #find how many true positives have been retrieved correctely
          maxTaken=Operations.jaccardSimilarity(self.A.intersection(self.B),self.A)
        lsh.b=i
        lsh.r=int(lsh.nHashes/i)
        lsh.lsh()
        b=lsh.findCosineSimilarsApproximated()
        jacc=lsh.jaccardAB()
        #if global optimum save values
        if jacc>maxJaccard:
          maxJaccard=jacc
          maxR=lsh.r
          maxB=lsh.b
          #find how many true positives have been retrieved correctely
          maxTaken=Operations.jaccardSimilarity(self.A.intersection(self.B),self.A)

      print([maxJaccard,maxR,maxB])
      print(maxTaken)
      self.r=maxR
      self.b=maxB
      return [maxJaccard,maxR,maxB]


this is the vectorized lsh class.
it generates the matrix m using vectorized operations with numpy and performs the lsh and exact similars functions in a vectorized ways.
This significately speed things up because operations are parallelized and optimized in c to speed things up.

In [54]:
#every operation of lsh has been vectorize here using numpy for better performances
class LshVectorized:

    def __init__(self,X,reviews,b=10,r=10,testPercentage=0.005,nHashes=100):
        self.X=X
        self.reviews=reviews
        self.rows=X.shape[0]
        self.cols=X.shape[1]
        self.b=b
        self.r=r
        self.testPercentage=testPercentage
        self.nHashes=nHashes
        self.splitDataset()

    def setBR(self,b,r):
        self.b=b
        self.r=r
    #split the dataset in train and test set
    def splitDataset(self):
        indexSplit=int(self.rows*self.testPercentage)
        self.testReviews,self.trainReviews=self.reviews[:indexSplit],self.reviews[indexSplit:]
        self.testX,self.trainX=self.X[:indexSplit],self.X[indexSplit:]
        self.rowsTest,self.colsTest=self.testX.shape
        self.rowsTrain,self.colsTrain=self.trainX.shape
    #generate the matrix of the signatures
    def generateMMM(self):
      #generate random normal distributed vectors U with shape n of cols of the document and number of hashes
      allU=np.random.normal(0,1,[self.cols,self.nHashes])
      #transformt into sparse matrix to make the next multiplication faster 
      allU=scipy.sparse.csr_matrix(allU)
      #multiply the test and train tfidf vectors with the u matrix and take the sign of the resulting matrix 
      #the obtained 2 matrices are the matrices M obtained with vectorized operations
      array=np.array((self.testX * allU).toarray())
      self.mTest=np.sign(array)
      array=np.array((self.trainX * allU).toarray())
      self.mTrain=np.sign(array)
    #lsh done using vectorized operations
    def lshMM(self):
      #create atrix of random numbers with dimension b, 30 that is the number of vectors for each hash and r
      hashes=np.random.randn(self.b,30,self.r)
      self.hashesMM=hashes
      #take the firsts r*b elements of m and those will compose the train and test signature 
      signatureTrain=self.mTrain[:,0:self.r*self.b].reshape(-1,self.b,self.r)
      #produce a matrix of size number of rows x b(number of hash functions)
      #i used the einsum operator and the hash generated with this expression for every element 
      #is the element multiplied by a matrix of signatures of size row x hashes and taking the sum of the entries of the resulting vector
      #in this way in each entry r b i will have a value corresponding to the key of the bucket in which it has to be hashed seeing it in the normal way      
      self.signatureTrain=np.einsum("abr, bdr-> ab",signatureTrain,hashes)
      #same for the test signatures
      signatureTest=self.mTest[:,0:self.r*self.b].reshape(-1,self.b,self.r)
      self.signatureTest=np.einsum("abr, bdr-> ab",signatureTest,hashes)
    #starting from the matrix of the signatures find the indices of the rows that are equal to the signature this is done in a vectorized way
    def findCosineSimilarsApproximatedMM(self):
        self.approximatedSimilars={}
        #for every signature test
        for i,signature in enumerate(self.signatureTest):
          #non zero returns rows and columns that satisfy the condition signature train equal to signature(signature is broadcasted on all the rows)
          #in this way at every key i i will have the set of row indices that have equal signature in the corresponding bucket(same column)
          self.approximatedSimilars[i]=set((self.signatureTrain==signature).nonzero()[0])
        return self.approximatedSimilars
    #does the same of the previous function but for one single value so it returns the indices of the elements that have the right signature
    def findCosineSimilarsApproximatedMMSingular(self,i):
        return set((self.signatureTrain==self.signatureTest[i,:]).nonzero()[0])
    #this function finds the elements with the exact cosine similarity in a vectorized way
    def findExactCosineSimilarsMM(self,threshold=0.5):
        self.exactSimilars={}
        #i create a matrix of the similaritis multiplisg the test vectors with the trian vectors trasposed and then normalizing with the norms
        sim=self.testX.dot(self.trainX.T)
        #i take the norms of the vectors of norm and test
        normTest=scipy.sparse.linalg.norm(self.testX,axis=1)
        normTrain=scipy.sparse.linalg.norm(self.trainX,axis=1)
        #i divide sim by the norm of train and for the norm of test transposed and find the indices ( rows and columns) of the elements
        # with a selected thresold 
        sim=sim/normTrain
        sim=(sim.T/normTest).T
        similars=(sim>=threshold).nonzero()
        #i create the dictionary of the similars for every element of the test set
        for i in range(len(similars[0])):
          if similars[0][i] not in self.exactSimilars:
            self.exactSimilars[similars[0][i]]=set()
          self.exactSimilars[similars[0][i]].add(similars[1][i])
        return self.exactSimilars
    # function used to find the optimal values of r and b given cosine similarity and n of hashes
    def optimizeRBMM(self,cosineSimilarity=0.2,nHashes=10):
      self.findExactCosineSimilarsMM(cosineSimilarity)
      self.nHashes=nHashes
      maxTaken=0
      lsh.generateMMM()
      #automatic search for b and r
      maxJaccard=0
      maxR=0
      maxB=0
      for i in range(1,int(lsh.nHashes**0.5)):
        lsh.r=i
        lsh.b=int(lsh.nHashes/i)
        lsh.lshMM()
        b=lsh.findCosineSimilarsApproximatedMM()
        jacc=lsh.jaccardAB()
        if jacc>maxJaccard:
          maxJaccard=jacc
          maxR=lsh.r
          maxB=lsh.b
          maxTaken=Operations.jaccardSimilarity(self.A.intersection(self.B),self.A)
        lsh.b=i
        lsh.r=int(lsh.nHashes/i)
        lsh.lshMM()
        b=lsh.findCosineSimilarsApproximatedMM()
        jacc=lsh.jaccardAB()
        if jacc>maxJaccard:
          maxJaccard=jacc
          maxR=lsh.r
          maxB=lsh.b
          maxTaken=Operations.jaccardSimilarity(self.A.intersection(self.B),self.A)
      print([maxJaccard,maxR,maxB])
      print(maxTaken)
      self.r=maxR
      self.b=maxB
      return [maxJaccard,maxR,maxB]
    #create the sets a and b
    def createAB(self):
        self.A=set()
        self.B=set()
        for i in self.exactSimilars:
            for j in self.exactSimilars[i]:
                self.A.add((i,j))
        for i in self.approximatedSimilars:
            for j in self.approximatedSimilars[i]:
                self.B.add((i,j))
    #finds jaccard similarity of exact and approximated similars couples
    def jaccardAB(self):
        return Operations.jaccardSimilarity(self.A,self.B)


start by taking a percentage of the dataset if you want to test the functions on just a portion of the dataset before using everything on the entire dataset.
If you use the non vectorized lsh there is the need to take a small percentage of the dataset otherwise it will take too much time because python is slow if it doesnt use parallelized and optimized libraries 

In [55]:
reviews,X=readReviewsX()
reviews,X=percentageOfDataset(reviews,X,0.2)
rows,cols=X.shape


here you can use the vectorized lsh

In [56]:
lsh=LshVectorized(X,reviews)


r and b can be selected and used to generate M and to calculate the lsh

In [57]:
lsh.r=5
lsh.b=5
lsh.nHashes=lsh.r*lsh.b
lsh.generateMMM()
lsh.lshMM()

passing the index of a test vector we can see all the similars to it in the training set

In [63]:
lsh.findCosineSimilarsApproximatedMMSingular(1)
"finished"

'finished'

this function calculates the exact similars for the entire test set use it only on a portion of the dataset otherwise the session will crash because it uses a lot of memory but is ultra fast

In [ ]:
lsh.findExactCosineSimilarsMM(0.3)
"finished"

use this function to calculate all the cosine similaras for every vector in the test set use it on a portion of the dataset or when not too many vectors are found otherwise too much memory is used

In [65]:
lsh.findCosineSimilarsApproximatedMM()
"finished"

'finished'

this function creates the a and b sets as in the homework description as sets of tuples

In [66]:
lsh.createAB()
"finished"

'finished'

print the dimensions of the 2 sets their differenceBA( how many false positive ), their intersection ( positives correctively found), jaccard and differenceAB(how many false negatives)

In [67]:
print(f"dima={len(lsh.A)}, dimB={len(lsh.B)} , differenceBA={len(lsh.B.difference(lsh.A))} , intersection={len(lsh.B.intersection(lsh.A))} , jaccard={lsh.jaccardAB()} , differenceAB={len(lsh.A.difference(lsh.B))}")

dima=26009, dimB=10256217 , differenceBA=10246294 , intersection=9923 , jaccard=0.0009659956486875436 , differenceAB=16086


those two functions have been useful to select b and r and to see if everything was working correctly (analyzing the intersection of a and b / a wrt the function  y=1-(1-x^r)^b  that gives the probability of an object with similarity x being found by lsh

function that starting from cosine similarity gives what p(x==y) approximates

In [ ]:
fromsim=0.4
top=1-np.arccos(fromsim)/np.pi
top

0.6309898804344547

function that starting from p(x==y) gives the cosine similarity
from the funciton

In [ ]:
fromp=0.6
tosim=np.cos((1-fromp)*np.pi)
tosim

0.30901699437494745